# YOLO split faces in each picture and OpenCV reshape image to fixed size

## Splitting Faces [(Repo 1](https://github.com/sthanhng/yoloface), [Repo 2)](https://github.com/thangnch/yolo_beginner)

In [0]:
# IMPORT LIB

import cv2
import numpy as np
import requests as req

In [0]:
IMAGE = "/home/shanenoi/Pictures/bo-anh-cuoi-tuyet-dep-tai-hien-sai-gon-nhung-nam-80-38-.1946.jpg"

In [0]:
class FaceSplitting(object):

    CONFIG = "/media/shanenoi/WINDOWS <-> LINUX/Hot-Girl-Detection/yoloface-master/cfg/yolov3-face.cfg"
    WEIGHTS = "/media/shanenoi/WINDOWS <-> LINUX/Hot-Girl-Detection/yoloface-master/model-weights/yolov3-wider_16000.weights"
    SCALE = 0.00392
    CONF_THRESHOLD = 0.5
    NMS_THRESHOLD = 0.4

    def __init__(self):
        self.image = None
        self.confidences = []
        self.boxes = []

    def load_config(self) -> None:
        self.net = cv2.dnn.readNet(self.WEIGHTS, self.CONFIG)

    def load_image(self, image) -> None:
        self.image = cv2.imread(image)
        self.blob = cv2.dnn.blobFromImage(self.image, self.SCALE, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(self.blob)

    def splitting(self) -> list:
        outs = self.net.forward(self.get_output_layers(self.net))
        Width = self.image.shape[1]
        Height = self.image.shape[0]

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    self.confidences.append(float(confidence))
                    self.boxes.append([x, y, w, h])

        indices = cv2.dnn.NMSBoxes(self.boxes, self.confidences, self.CONF_THRESHOLD, self.NMS_THRESHOLD)

        for i in indices:
            i = i[0]
            box = self.boxes[i]
            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            yield {
                'x': round(x),
                'y': round(y),
                'x+w': round(x+w),
                'y+h': round(y+h)
            }

    @staticmethod
    def get_output_layers(net) -> list:
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        print(type(output_layers))
        return output_layers